# `II. Field Simulation` 
This .pynb file reads `./inter_results/mesh_result.pkl` generated by `./I_Mesh_Processing.ipynb`, and simulates electric field distribution. The result is stored in `./inter_results/field_result.pkl`.

In this file we compute electric field or potential on some spatial points(called `grid`) for all electrode voltage configurations(i.e. one eletrode is 1V and the rest 0V).Major calculations calls `fastlap` C library which uses a pre-conditioned, adaptive, multipole-accelerated algorithm for solving Laplace problem. Two parameters control multipole acceleration.
+ num_mom, the number of multipole
+ num_lev, the number of levels in the hierarchical spatial decomposition.  
num_lev=1 means direct computation without multipole acceleration. See fastlap ug.pdf and README.rst.


In `run_job` function, `job` is `Configuration` instance and `grid` is discretirized spatial grid (not the mesh). The general workflow (also the routine of BEM method) are:  
1. `solve_singularities()` solves charge distributions by iterative methods to make it consistent with one electrode at 1V and others at 0V (unit potentials). `adapt_mesh()` refines meshes adaptively to achieve certain precision while solving sigulartities.
2. Compute potentials on given grid points by `simulate()`, based on the charge distributions gotten previously.
3. Potential data of each unit potential are saved seperately to a `Result` instance, and also export to VTK files.
4. Return total accumulated charge per electrode in the end.

notes:
* here we invoke `multiprocessing.set_start_method("fork")` for compatibility with python version 3.9
* This is the most time consuming part in our workflow. Running following codes in a `.py` file instead of `.ipynb` may be helpful. Closing other apps in your laptop is helpful. Using HPC is also an option. 
* for reference, this file in my laptop(macbook pro, 2 physical kernels) runs 800s
* This `.ipynb` file will generate a `./.vtk` file for intermediate files, which can be simply ignored.

## (1) import the python packages needed for the script



In [1]:
from time import time
import pickle
import numpy as np
import os
import pyvista as pv
import sys
import vtk
sys.path.append('/home/sqip/Documents/github/bem')
from bem import Electrodes, Sphere, Mesh, Grid, Configuration, Result
import multiprocessing 
# multiprocessing.set_start_method("fork")
from utils.helper_functions import *

## (2) import the mesh saved from I_Mesh_Processing
This should correspond to the same name was exported during mesh_processing. The reason this is even an option is in case you want to load in a different saved mesh. 

In [2]:
radius= 500e-3
area = 2e-4
file = 'htrap'
file_in_name = 'inter_results/htrap/'+file+'_'+str(radius)+'_'+str(area)+'.pkl'
vtk_out = "inter_results/htrap/.vtks/"+file
file_out_name = 'inter_results/htrap/'+file+'_'+str(radius)+'_'+str(area)+'_simulation'


#open the mesh that will be used for simulation
with open(file_in_name,'rb') as f:
    mesh_unit,xl,yl,zl,mesh,electrode_names= pickle.load(f) # import results from mesh processing

## (2) define grid
Here, we define some spatial points for simulation field, called 'grid'.
The grid is typically a 3-d cube centered on the trapping location.
* s is the unit step for the spatial points
* Lx, Ly, Lz are the dimensions of the 3-d cube
* sx, sy, sz are the step size along each dimension
* nx, ny, nz are the number of points to be simulated along each dimension
* `Grid` is the function imported from the bem package that will be inputted into the simluation

In [3]:
with open(file_in_name,'rb') as f:
    mesh_unit,xl,yl,zl,mesh,electrode_names= pickle.load(f) # import results from mesh processing
# grid to evalute potential and fields atCreate a grid in unit of scaled length mesh_unit. Only choose the interested region (trap center) to save time.
Lx, Ly, Lz = 20*1e-3,20*1e-3,20*1e-3# in the unit of scaled length mesh_unit
# xl,yl,zl = -3.75*1e-3,72*1e-3,270*1.0e-3
xl,yl,zl = 3.0e-3,75.0e-3,0.0e-3
s = 2e-3
sx,sy,sz = s,s,s
print("done")
# ni is number of grid points, si is step size. To  on i direction you need to fix ni*si.
nx, ny, nz = [int(Lx/sx),int(Ly/sy),int(Lz/sz)]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(xl,yl,zl), step=(sx, sy, sz), shape=(nx,ny,nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("lowval",grid.indices_to_coordinates([0,0,0]))
print("Grid center index", grid.indices_to_coordinates((nx/2,ny/2,nz/2)))
print("gridpts:",nx*ny*nz)
center = (xl,yl,zl)
step = (sx,sy,sz)
shape = (nx,ny,nz)
# xyz = np.array([np.linspace(c-s*(h-1)/2., c+s*(h-1)/2.,h)
#                 for c, s, h in zip(center, step, shape)])

done
Size/l: 0.02 0.02 0.02
Step/l: 0.002 0.002 0.002
Shape (grid point numbers): 10 10 10
lowval [-0.006  0.066 -0.009]
Grid center index [0.004 0.076 0.001]
gridpts: 1000


## (2) run jobs
`pmap.multiprocessing.Pool(processes).map(job,list)` is used to do several simulations in parallel (depends what you define as parallel). 

`Pool(number)` creates a set of worker processes, called a pool, to submit tasks to. When a job is submitted to a worker process, the worker process executes that task. <b>Each worker process is associated with its own CPU core</b> which means the number of worker processes in the pool is the number of parallel computations that can be performed. <b>number</b> is the number of worker processes to start. 

In other words, if you want to use only 1 core at a time, write Pool(1), if you want 2 cores write Pool(2). If no number is given, `Pool` will create as many worker processes as there are cores on the computer. This speeds things up (you can use all cores simulatneously), but it is important to point out that since each worker process will be requesting RAM, this will consume more memory on your computer than Pool(1). If the total RAM requested exceeds what your computer has, <b>you're gonna have a bad time</b>. One of two things will likely happen- first is that your computer will try to start using 'swap space' which is using disk memory as RAM which will slow down the computation significantly- this is not obvious unless you are looking at the resources being used by your computer (look for normal memory used and swap memory used on system monitor). Second is that your computer wil crash because the worker processes have consumed all the memory. 

`Pool.map`(<b>job</b>,<b>jobs_list</b>) takes two arguments:
* <b>job</b> is the python function that will be turned into a job to be executed by the worker processes
* <b>jobs_list</b> is a list where each item in the list is a set of arguments to be provided to the function when creating a job

The worker processes then execute any job sent to the pool. 

In [4]:
jobs = list(Configuration.select(mesh,'DC.*','RF'))    # select() picks one electrode each time.
# run the different electrodes on the parallel pool
pmap = multiprocessing.Pool(4).map # parallel map
#pmap = map # serial map
t0 = time()
# range(len(jobs))
def run_map():
    pmap(run_job, ((jobs[i], grid, vtk_out,i,len(jobs)) for i in np.arange(len(jobs))))
    print("Computing time: %f s"%(time()-t0))
    # run_job casts a word after finishing ea"ch electrode.

run_map()

starting job 1 out of 22
starting job 5 out of 22
starting job 3 out of 22
starting job 7 out of 22
finished job DC2
starting job 4 out of 22


2023-11-30 10:31:19.533 ( 173.154s) [         115033D]      vtkDataWriter.cxx:176    ERR| vtkPolyDataWriter (0x7fc478c04080): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC2_mesh.vtk
ERROR:root:Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC2_mesh.vtk
/Users/Ben/opt/anaconda3/envs/bem39/lib/python3.9/site-packages/pyvista/core/grid.py:549: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(
2023-11-30 10:31:19.541 ( 173.163s) [         115033D]      vtkDataWriter.cxx:176    ERR| vtkStructuredPointsWriter (0x7fc4c89e13b0): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC2.vtk
E

finished job DC13
starting job 8 out of 22


2023-11-30 10:31:21.389 ( 174.974s) [         1150345]      vtkDataWriter.cxx:176    ERR| vtkPolyDataWriter (0x7f9efbffd160): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC13_mesh.vtk
ERROR:root:Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC13_mesh.vtk
/Users/Ben/opt/anaconda3/envs/bem39/lib/python3.9/site-packages/pyvista/core/grid.py:549: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(
2023-11-30 10:31:21.394 ( 174.979s) [         1150345]      vtkDataWriter.cxx:176    ERR| vtkStructuredPointsWriter (0x7f9e8c905080): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC13.vt

finished job DC11
starting job 6 out of 22


...

2023-11-30 10:39:47.982 ( 681.566s) [         1150345]      vtkDataWriter.cxx:176    ERR| vtkPolyDataWriter (0x7f9efc8afa80): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC5_mesh.vtk
ERROR:root:Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC5_mesh.vtk
2023-11-30 10:39:47.983 ( 681.567s) [         1150345]      vtkDataWriter.cxx:176    ERR| vtkStructuredPointsWriter (0x7f9efc8a0620): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC5.vtk
ERROR:root:Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC5.vtk


finished job DC8


2023-11-30 10:39:57.643 ( 691.254s) [         1150341]      vtkDataWriter.cxx:176    ERR| vtkPolyDataWriter (0x7fcb13206140): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC8_mesh.vtk
ERROR:root:Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC8_mesh.vtk
2023-11-30 10:39:57.644 ( 691.255s) [         1150341]      vtkDataWriter.cxx:176    ERR| vtkStructuredPointsWriter (0x7fcb13208e60): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC8.vtk
ERROR:root:Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC8.vtk


finished job DC10
Computing time: 695.919744 s


2023-11-30 10:40:01.625 ( 695.256s) [         1150339]      vtkDataWriter.cxx:176    ERR| vtkPolyDataWriter (0x7f95bd6060e0): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC10_mesh.vtk
ERROR:root:Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC10_mesh.vtk
2023-11-30 10:40:01.625 ( 695.257s) [         1150339]      vtkDataWriter.cxx:176    ERR| vtkStructuredPointsWriter (0x7f95bd608e00): Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC10.vtk
ERROR:root:Unable to open file: /Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC10.vtk


## (3) view simulation results
The results can be viewed in an interactive 3-d plot using the package pyvista

In [5]:
"the last simulated mesh was for mesh radius 500e-3mm and max triangle area 3.125e-5 mm"
ele = 'DC3'
data_name = "%s_%s.vtk" % (vtk_out, ele)
data = pv.UniformGrid(data_name)
scalar_name = 'potential'
avals = data[scalar_name]
range = avals.max()-avals.min()
input_range = [avals.min(),avals.min()+range]
Result.view(vtk_out, ele) # add electrode name between '' for result view

FileNotFoundError: File (/Users/Ben/Library/Mobile Documents/com~apple~CloudDocs/GitHub/https:/github.com/haeffnerlab/bem/bem/examples/sqip/inter_results/htrap/.vtks/htrap_DC3.vtk) not found

In [ ]:
# for x in np.arange(0,4):
data_name = "%s_%s.vtk" % (vtk_out, 'RF')
import pyvista as pv 
data = pv.UniformGrid(data_name)
scalar_name = 'potential'

avals = data[scalar_name]
input_range = [-0.2,5.0]
Result.view(vtk_out, 'RF') # add electrode name between '' for result view

/Users/Ben/opt/anaconda3/envs/bem39/lib/python3.9/site-packages/pyvista/core/grid.py:912: PyVistaDeprecationWarning: `UniformGrid` is deprecated. Use `ImageData` instead.
  warnings.warn(


## (4) save simulation results

In [ ]:
electrode_names = ['DC1','DC2','DC3','DC4','DC5','DC6','DC7','DC8','DC9','DC10',
                   'DC11','DC12','DC13','DC14','DC15','DC16','DC17','DC18','DC19','DC20','DC21',
                   'RF']
write_pickle(vtk_out,file_out_name,grid,electrode_names)